# Pre-processing

Project :focus on the numeracy part only.

In [2]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import re

sns.set_style('whitegrid')
#sns.set_palette("husl")

%matplotlib inline

In [3]:
# train data
y7_2012_y9_2014 = pd.read_csv('y7_2012_y9_2014_2.csv' )
y7_2013_y9_2015 = pd.read_csv('y7_2013_y9_2015_2.csv' )
y7_2014_y9_2016 = pd.read_csv('y7_2014_y9_2016_2.csv' )
# test data
y7_2015_y9_2017 = pd.read_csv('y7_2015_y9_2017_2.csv' )
# need to predict
y7_2016 = pd.read_csv('./NAPLAN/naplan_schoolexportYr7_2016.csv')
y7_2017 = pd.read_csv('./NAPLAN/naplan_schoolexportYr7_2017.csv')

In [4]:
# import questions, only import the columns: 'Question_number','Description','Syllabus'
question_y7_2012 = pd.read_csv('./NAPLAN_Questions/tabula-Questions Year 7 2012.csv').iloc[:,:3]
question_y7_2012.dropna(inplace = True)
question_y7_2012.reset_index(inplace = True, drop = True)
question_y7_2013 = pd.read_csv('./NAPLAN_Questions/tabula-Questions Year 7 2013.csv').iloc[:,:3]
question_y7_2013.dropna(inplace = True)
question_y7_2013.reset_index(inplace = True, drop = True)
question_y7_2014 = pd.read_csv('./NAPLAN_Questions/tabula-Questions Year 7 2014.csv').iloc[:,:3]
question_y7_2014.dropna(inplace = True)
question_y7_2014.reset_index(inplace = True, drop = True)
question_y7_2015 = pd.read_csv('./NAPLAN_Questions/tabula-Questions Year 7 2015.csv').iloc[:,:3]
question_y7_2015.dropna(inplace = True)
question_y7_2015.reset_index(inplace = True, drop = True)
question_y7_2016 = pd.read_csv('./NAPLAN_Questions/tabula-Questions Year 7 2016.csv').iloc[:,:3]
question_y7_2016.dropna(inplace = True)
question_y7_2016.reset_index(inplace = True, drop = True)
question_y7_2017 = pd.read_csv('./NAPLAN_Questions/tabula-Questions Year 7 2017.csv').iloc[:,:3]
question_y7_2017.dropna(inplace = True)
question_y7_2017.reset_index(inplace = True, drop = True)


columns_name = ['Question_number','Description','Syllabus','Difficulty','Band',
                'School_%Correct','Group_%Correct','Group_Difference',
                'State_%Correct','State_Difference']

In [5]:
# create a verb columns
verbs_description = pd.DataFrame(question_y7_2012.Description.str.split(': ',1).tolist(),columns = ['a','b'])
question_y7_2012['Verbs'] = pd.DataFrame(verbs_description.b.str.split(' ',1).tolist(),columns=['Verbs','c']).Verbs

verbs_description = pd.DataFrame(question_y7_2013.Description.str.split(': ',1).tolist(),columns = ['a','b'])
question_y7_2013['Verbs'] = pd.DataFrame(verbs_description.b.str.split(' ',1).tolist(),columns=['Verbs','c']).Verbs

question_y7_2014['Verbs'] = pd.DataFrame(question_y7_2014.Description.str.split(' ',1).tolist(),columns=['Verbs','c']).Verbs

verbs_description = pd.DataFrame(question_y7_2015.Description.str.split(': ',1).tolist(),columns = ['a','b'])
question_y7_2015['Verbs'] = pd.DataFrame(verbs_description.b.str.split(' ',1).tolist(),columns=['Verbs','c']).Verbs

verbs_description = pd.DataFrame(question_y7_2016.Description.str.split(': ',1).tolist(),columns = ['a','b'])
question_y7_2016['Verbs'] = pd.DataFrame(verbs_description.b.str.split(' ',1).tolist(),columns=['Verbs','c']).Verbs

question_y7_2017['Verbs'] = pd.DataFrame(question_y7_2017.Description.str.split(' ',1).tolist(),columns=['Verbs','c']).Verbs

In [6]:
columns_name = ['Question_number','Description','Syllabus','Verbs']
question_y7_2012.columns = [c +'_2012' for c in columns_name]
question_y7_2013.columns = [c +'_2013' for c in columns_name]
question_y7_2014.columns = [c +'_2014' for c in columns_name]
question_y7_2015.columns = [c +'_2015' for c in columns_name]
question_y7_2016.columns = [c +'_2016' for c in columns_name]
question_y7_2017.columns = [c +'_2017' for c in columns_name]

## ** a new syllabus is used starting from 2013.**

In [7]:
question_y7_2012.head()

,Question_number_2012,Description_2012,Syllabus_2012,Verbs_2012
0,NC-01,3D: Determines the number of faces on an irreg...,SGS2.1,Determines
1,CA-01,Frac and Dec: Solves a word problem involving ...,NS3.4,Solves
2,NC-02,P and A: Calculates the value of a pronumeral ...,PAS3.1B,Calculates
3,CA-02,Integers: Calculates rise in temperature from ...,NS4.2,Calculates
4,NC-03,Time: Calculates elapsed time in minutes,MS3.5,Calculates


In [8]:
question_y7_2013.head()

,Question_number_2013,Description_2013,Syllabus_2013,Verbs_2013
0,NC-01,3D: Identifies the net of a pyramid.,MA3-14MG,Identifies
1,CA-01,Add and Sub: Subtracts two numbers each with 7...,MA3-5NA,Subtracts
2,NC-02,Data: Uses coordinates to identify the content...,MA3-17MG,Uses
3,CA-02,2D: Determines a line of symmetry.,MA3-15MG,Determines
4,NC-03,Data: Finds the maximum point on a frequency/t...,MA3-18SP,Finds


In [9]:
# a new syllabus is used starting from 2013.
# year 2012 has a different syllabus code. update it.
syllabus_dict_2012 = { 'DS3.1':'MA3-18SP', 'MS3.2':'MA3-10MG', 'MS3.3':'MA3-11MG', 'MS3.5':'MA3-13MG',
 'MS4.3':'MA4-15MG', 'NS2.3':'MA2-6NA', 'NS2.5':'MA2-19SP', 'NS3.1':'MA3-4NA', 'NS3.3':'MA3-6NA',
 'NS3.4':'MA3-7NA', 'NS3.4 NS3.5':'MA3-7NA', 'NS3.5':'MA3-19SP',
 'NS4.2':'MA4-4NA', 'NS4.3':'MA4-5NA', 'NS4.4':'MA4-21SP',
 'PAS3.1A':'MA3-8NA', 'PAS3.1B':'MA3-8NA','PAS4.2' :'MA3-8NA',
 'PAS4.4':'MA4-10NA', 'PAS4.5':'MA4-11NA', 'SGS2.1':'MA2-14MG', 'SGS2.2A':'MA2-15MG', 'SGS3.1':'MA3-14MG' ,
 'SGS3.2A':'MA3-15MG' , 'SGS3.2B':'MA3-16MG','SGS3.3':'MA3-17MG' ,'SGS4.3':'MA4-17MG'}

question_y7_2012['Syllabus_2012'] = question_y7_2012.Syllabus_2012.apply(syllabus_dict_2012.get)


In [10]:
question_y7_2012.head()

,Question_number_2012,Description_2012,Syllabus_2012,Verbs_2012
0,NC-01,3D: Determines the number of faces on an irreg...,MA2-14MG,Determines
1,CA-01,Frac and Dec: Solves a word problem involving ...,MA3-7NA,Solves
2,NC-02,P and A: Calculates the value of a pronumeral ...,MA3-8NA,Calculates
3,CA-02,Integers: Calculates rise in temperature from ...,MA4-4NA,Calculates
4,NC-03,Time: Calculates elapsed time in minutes,MA3-13MG,Calculates


## create the dataframe with only numeracy questions

In [11]:
X_y7_2012 = y7_2012_y9_2014[['NM7(C1)', 'NM7(NC1)', 'NM7(C2)', 'NM7(NC2)', 'NM7(NC3)',
 'NM7(C3)', 'NM7(NC4)', 'NM7(C4)', 'NM7(NC5)', 'NM7(C5)', 'NM7(NC6)',
 'NM7(C6)', 'NM7(C7)', 'NM7(NC7)', 'NM7(NC8)', 'NM7(C8)', 'NM7(NC9)', 
 'NM7(C9)', 'NM7(C10)', 'NM7(NC10)', 'NM7(NC11)', 'NM7(C11)', 'NM7(C12)', 
 'NM7(NC12)', 'NM7(C13)', 'NM7(NC13)', 'NM7(NC14)', 'NM7(C14)', 'NM7(C15)',
 'NM7(NC15)', 'NM7(C16)', 'NM7(NC16)', 'NM7(C17)', 'NM7(NC17)', 'NM7(NC18)', 
 'NM7(C18)', 'NM7(NC19)', 'NM7(C19)', 'NM7(NC20)', 'NM7(C20)', 'NM7(NC21)', 
 'NM7(C21)', 'NM7(C22)', 'NM7(NC22)', 'NM7(NC23)', 'NM7(C23)', 'NM7(C24)', 
 'NM7(NC24)', 'NM7(NC25)', 'NM7(C25)', 'NM7(C26)', 'NM7(NC26)', 'NM7(C27)', 
 'NM7(NC27)', 'NM7(C28)', 'NM7(NC28)', 'NM7(NC29)', 'NM7(C29)', 'NM7(C30)', 
 'NM7(NC30)', 'NM7(C31)', 'NM7(NC31)', 'NM7(C32)', 'NM7(C32).1','SRN']].set_index('SRN')



In [12]:
X_y7_2013 = y7_2013_y9_2015[['NM7(NC1)', 'NM7(C1)', 'NM7(C2)', 'NM7(NC2)', 'NM7(C3)', 'NM7(NC3)', 'NM7(NC4)', 'NM7(C4)', 
              'NM7(NC5)', 'NM7(C5)', 'NM7(NC6)', 'NM7(C6)', 'NM7(NC7)', 'NM7(C7)', 'NM7(C8)', 'NM7(NC8)', 
              'NM7(NC9)', 'NM7(C9)', 'NM7(NC10)', 'NM7(C10)', 'NM7(C11)', 'NM7(NC11)', 'NM7(C12)', 'NM7(NC12)', 
              'NM7(C13)', 'NM7(NC13)', 'NM7(NC14)', 'NM7(C14)', 'NM7(NC15)', 'NM7(C15)', 'NM7(C16)', 'NM7(NC16)', 
              'NM7(C17)', 'NM7(NC17)', 'NM7(NC18)', 'NM7(C18)', 'NM7(NC19)', 'NM7(C19)', 'NM7(NC20)', 'NM7(C20)', 
              'NM7(C21)', 'NM7(NC21)', 'NM7(NC22)', 'NM7(C22)', 'NM7(NC23)', 'NM7(C23)', 'NM7(C24)', 'NM7(NC24)', 
              'NM7(C25)', 'NM7(NC25)', 'NM7(C26)', 'NM7(NC26)', 'NM7(C27)', 'NM7(NC27)', 'NM7(C28)', 'NM7(NC28)', 
              'NM7(NC29)', 'NM7(C29)', 'NM7(NC30)', 'NM7(C30)', 'NM7(NC31)', 'NM7(C31)', 'NM7(NC32)', 'NM7(C32)','SRN']].set_index('SRN')


In [13]:
X_y7_2014 = y7_2014_y9_2016[['NM7(C1)', 'NM7(NC1)', 'NM7(C2)', 'NM7(NC2)', 'NM7(C3)', 
                             'NM7(NC3)', 'NM7(NC4)', 'NM7(C4)', 'NM7(C5)', 'NM7(NC5)', 
                             'NM7(C6)', 'NM7(NC6)', 'NM7(C7)', 'NM7(NC7)', 'NM7(NC8)', 
                             'NM7(C8)', 'NM7(C9)', 'NM7(NC9)', 'NM7(C10)', 'NM7(NC10)', 
                             'NM7(NC11)', 'NM7(C11)', 'NM7(NC12)', 'NM7(C12)', 'NM7(NC13)', 'NM7(C13)', 'NM7(NC14)', 'NM7(C14)', 'NM7(NC15)', 'NM7(C15)', 'NM7(C16)', 'NM7(NC16)', 'NM7(NC17)', 'NM7(C17)', 'NM7(NC18)', 'NM7(C18)', 'NM7(C19)', 'NM7(NC19)', 'NM7(NC20)', 'NM7(C20)', 'NM7(NC21)', 'NM7(C21)', 'NM7(NC22)', 'NM7(C22)', 'NM7(NC23)', 'NM7(C23)', 'NM7(C24)', 'NM7(NC24)', 'NM7(NC25)', 'NM7(C25)', 'NM7(NC26)', 'NM7(C26)', 'NM7(NC27)', 'NM7(C27)', 'NM7(C28)', 'NM7(NC28)', 'NM7(C29)', 'NM7(NC29)', 'NM7(C30)', 
                             'NM7(NC30)', 'NM7(C31)', 'NM7(NC31)', 'NM7(NC32)', 'NM7(C32)','SRN']].set_index('SRN')


In [14]:
X_y7_2015 = y7_2015_y9_2017[['NM7(C1)', 'NM7(NC1)', 'NM7(NC2)', 'NM7(C2)', 'NM7(NC3)', 
                             'NM7(C3)', 'NM7(C4)', 'NM7(NC4)', 'NM7(NC5)', 'NM7(C5)', 
                             'NM7(NC6)', 'NM7(C6)', 'NM7(C7)', 'NM7(NC7)', 'NM7(C8)', 
                             'NM7(NC8)', 'NM7(NC9)', 'NM7(C9)', 'NM7(NC10)', 'NM7(C10)',
                             'NM7(C11)', 'NM7(NC11)', 'NM7(NC12)', 'NM7(C12)', 'NM7(C13)', 
                             'NM7(NC13)', 'NM7(C14)', 'NM7(NC14)', 'NM7(NC15)', 'NM7(C15)',
                             'NM7(C16)', 'NM7(NC16)', 'NM7(C17)', 'NM7(NC17)', 'NM7(NC18)',
                             'NM7(C18)', 'NM7(NC19)', 'NM7(C19)', 'NM7(NC20)', 'NM7(C20)', 
                             'NM7(NC21)', 'NM7(C21)', 'NM7(NC22)', 'NM7(C22)', 'NM7(NC23)', 
                             'NM7(C23)', 'NM7(C24)', 'NM7(NC24)', 'NM7(NC25)', 'NM7(C25)', 
                             'NM7(NC26)', 'NM7(C26)', 'NM7(NC27)', 'NM7(C27)', 'NM7(NC28)',
                             'NM7(C28)', 'NM7(NC29)', 'NM7(C29)', 'NM7(C30)', 
                             'NM7(NC30)', 'NM7(NC31)', 'NM7(C31)', 'NM7(NC32)', 'NM7(C32)','SRN']].set_index('SRN')

In [15]:
X_y7_2016 = y7_2016[['NM7(C1)', 'NM7(NC1)', 'NM7(NC2)', 'NM7(C2)', 'NM7(NC3)', 
                     'NM7(C3)', 'NM7(NC4)', 'NM7(C4)', 'NM7(C5)', 'NM7(NC5)', 
                     'NM7(NC6)', 'NM7(C6)', 'NM7(C7)', 'NM7(NC7)', 'NM7(NC8)', 
                     'NM7(C8)', 'NM7(NC9)', 'NM7(C9)', 'NM7(NC10)', 'NM7(C10)', 
                     'NM7(C11)', 'NM7(NC11)', 'NM7(NC12)', 'NM7(C12)', 'NM7(NC13)',
                     'NM7(C13)', 'NM7(C14)', 'NM7(NC14)', 'NM7(NC15)', 'NM7(C15)',
                     'NM7(C16)', 'NM7(NC16)', 'NM7(NC17)', 'NM7(C17)', 
                     'NM7(NC18)', 'NM7(C18)', 'NM7(NC19)', 'NM7(C19)', 'NM7(NC20)',
                     'NM7(C20)', 'NM7(NC21)', 'NM7(C21)', 'NM7(NC22)', 'NM7(C22)',
                     'NM7(C23)', 'NM7(NC23)', 'NM7(NC24)', 'NM7(C24)', 'NM7(C25)', 
                     'NM7(NC25)', 'NM7(C26)', 'NM7(NC26)', 'NM7(C27)', 'NM7(NC27)', 
                     'NM7(NC28)', 'NM7(C28)', 'NM7(NC29)', 'NM7(C29)', 'NM7(NC30)',
        
                     'NM7(C30)', 'NM7(NC31)', 'NM7(C31)', 'NM7(C32)', 'NM7(NC32)','SRN']].set_index('SRN')


In [16]:
X_y7_2017 = y7_2017[['NM7(C1)', 'NM7(NC1)', 'NM7(C2)', 
                     'NM7(NC2)', 'NM7(C3)', 'NM7(NC3)', 
                     'NM7(NC4)', 'NM7(C4)', 'NM7(C5)', 
                     'NM7(NC5)', 'NM7(NC6)', 'NM7(C6)', 'NM7(NC7)', 'NM7(C7)', 'NM7(NC8)', 
                     'NM7(C8)', 'NM7(C9)', 'NM7(C10)', 'NM7(C11)', 'NM7(C12)', 'NM7(C13)', 
                     'NM7(C14)', 'NM7(C15)', 'NM7(C16)', 'NM7(C17)', 'NM7(C18)',
                     'NM7(C19)', 'NM7(C20)', 'NM7(C21)', 'NM7(C22)', 'NM7(C23)',
                     'NM7(C24)', 'NM7(C25)', 'NM7(C26)', 'NM7(C27)', 'NM7(C28)', 
                     
                     'NM7(C29)', 'NM7(C30)', 'NM7(C31)', 'NM7(C32)', 'NM7(C33)', 
                     'NM7(C34)', 'NM7(C35)', 'NM7(C36)', 'NM7(C37)', 'NM7(C38)', 
                     'NM7(C39)', 'NM7(C40)','SRN']].set_index('SRN')


## Replace the column names with the syllabus code

Create the features columns. With the same syllabus code, I can merge the data from different years.


####  Year 2012

In [95]:
# drop those students who has not attend the test
X_y7_2012.dropna(inplace=True)

In [23]:
X_y7_2012.head()

,NM7(C1),NM7(NC1),NM7(C2),NM7(NC2),NM7(NC3),NM7(C3),NM7(NC4),NM7(C4),NM7(NC5),NM7(C5),...,NM7(C28),NM7(NC28),NM7(NC29),NM7(C29),NM7(C30),NM7(NC30),NM7(C31),NM7(NC31),NM7(C32),NM7(C32).1
SRN,,,,,,,,,,,,,,,,,,,,,
436530790,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
441546149,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
432640256,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
434220173,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
431672294,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [25]:
question_y7_2012.Question_number_2012 = question_y7_2012.Question_number_2012.str.replace('-','')
question_y7_2012.Question_number_2012 = question_y7_2012.Question_number_2012.str.replace('A0','')
question_y7_2012.Question_number_2012 = question_y7_2012.Question_number_2012.str.replace('C0','C')
question_y7_2012.Question_number_2012 = question_y7_2012.Question_number_2012.str.replace('A','')

question_y7_2012.iloc[63].Question_number_2012 = 'C32.1'

In [26]:
column_name = pd.Series(X_y7_2012.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2012_1 = pd.DataFrame(column_name,columns=['Q'])

In [27]:
df2012_2 = pd.merge(df2012_1, question_y7_2012, left_on = 'Q', right_on = 'Question_number_2012',
        how = 'inner')

In [28]:
# rename the column names
X_y7_2012.columns = df2012_2.Syllabus_2012

In [29]:
# group by for the same syllabus and take the mean value

df2012_3 = X_y7_2012.T.reset_index().groupby(['Syllabus_2012']).apply(np.mean)

In [31]:
new_X_y7_2012 = df2012_3.T
new_X_y7_2012.head()

Syllabus_2012,MA2-14MG,MA2-15MG,MA2-19SP,MA2-6NA,MA3-10MG,MA3-11MG,MA3-13MG,MA3-14MG,MA3-15MG,MA3-16MG,...,MA3-6NA,MA3-7NA,MA3-8NA,MA4-10NA,MA4-11NA,MA4-15MG,MA4-17MG,MA4-21SP,MA4-4NA,MA4-5NA
SRN,,,,,,,,,,,,,,,,,,,,,
436530790,1.0,0.8,1.0,1.0,0.5,0.0,0.666667,0.5,1.0,0.5,...,1.0,0.833333,0.6,1.0,0.0,0.0,0.0,0.0,1.0,0.222222
441546149,1.0,0.8,1.0,1.0,0.5,0.0,0.666667,0.5,1.0,1.0,...,1.0,1.000000,0.8,1.0,1.0,1.0,0.0,1.0,1.0,0.666667
432640256,1.0,0.6,1.0,1.0,0.0,0.0,1.000000,1.0,1.0,0.5,...,1.0,0.666667,0.3,0.0,0.0,0.0,1.0,1.0,1.0,0.555556
434220173,1.0,1.0,1.0,1.0,0.5,0.0,0.666667,0.5,1.0,0.5,...,0.5,0.666667,0.5,1.0,0.0,0.0,0.0,1.0,1.0,0.222222
431672294,1.0,1.0,1.0,1.0,0.5,0.0,0.666667,1.0,1.0,0.5,...,1.0,1.000000,0.8,1.0,0.0,1.0,0.0,0.0,1.0,0.777778


####  Year 2013

In [32]:
column_name = pd.Series(X_y7_2013.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2013_1 = pd.DataFrame(column_name,columns=['Q'])

In [33]:
question_y7_2013.Question_number_2013 = question_y7_2013.Question_number_2013.str.replace('-','')
question_y7_2013.Question_number_2013 = question_y7_2013.Question_number_2013.str.replace('A0','')
question_y7_2013.Question_number_2013 = question_y7_2013.Question_number_2013.str.replace('C0','C')
question_y7_2013.Question_number_2013 = question_y7_2013.Question_number_2013.str.replace('A','')


In [34]:
df2013_2 = pd.merge(df2013_1, question_y7_2013, left_on = 'Q', right_on = 'Question_number_2013',
        how = 'outer')

In [35]:
df2013_2.isnull().any()

Q                       False
Question_number_2013    False
Description_2013        False
Syllabus_2013           False
Verbs_2013              False
dtype: bool

In [37]:
X_y7_2013.columns = df2013_2.Syllabus_2013

In [38]:
X_y7_2013.head()

Syllabus_2013,MA3-14MG,MA3-5NA,MA3-15MG,MA3-17MG,MA3-14MG,MA3-18SP,MA2-5NA,MA3-14MG,MA4-5NA,MA4-5NA,...,MA4-5NA,MA3-18SP,MA4-7NA,MA4-7NA,MA4-8NA,MA4-6NA,MA4-7NA,MA4-7NA,MA3-9MG,MA3-9MG
SRN,,,,,,,,,,,,,,,,,,,,,
436607369,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
429152038,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
434564981,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
428846257,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
429541042,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0


In [39]:
df2013_3 = X_y7_2013.T.reset_index().groupby(['Syllabus_2013']).apply(np.mean)

In [40]:
new_X_y7_2013 = df2013_3.T

In [41]:
new_X_y7_2013.head()

Syllabus_2013,MA2-12MG,MA2-13MG,MA2-17MG,MA2-5NA,MA2-8NA,MA3-11MG,MA3-14MG,MA3-15MG,MA3-16MG,MA3-17MG,...,MA4-13MG,MA4-15MG,MA4-17MG,MA4-20SP,MA4-21SP,MA4-4NA,MA4-5NA,MA4-6NA,MA4-7NA,MA4-8NA
SRN,,,,,,,,,,,,,,,,,,,,,
436607369,1.0,0.0,1.0,1.0,1.0,1.0,0.8,0.666667,1.0,1.0,...,0.0,1.0,0.5,0.0,0.0,1.0,0.714286,0.0,0.500000,1.0
429152038,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.666667,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.714286,0.0,0.833333,1.0
434564981,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.857143,0.0,0.833333,1.0
428846257,1.0,1.0,0.0,1.0,1.0,1.0,0.8,0.666667,1.0,1.0,...,0.0,1.0,0.5,1.0,1.0,1.0,0.714286,1.0,0.166667,1.0
429541042,1.0,1.0,1.0,1.0,1.0,0.0,0.6,0.666667,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.428571,0.0,0.666667,1.0


# 2014

In [42]:
column_name = pd.Series(X_y7_2014.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2014_1 = pd.DataFrame(column_name,columns=['Q'])

In [43]:
question_y7_2014.Question_number_2014 = question_y7_2014.Question_number_2014.str.replace('-','')
question_y7_2014.Question_number_2014 = question_y7_2014.Question_number_2014.str.replace('A0','')
question_y7_2014.Question_number_2014 = question_y7_2014.Question_number_2014.str.replace('C0','C')
question_y7_2014.Question_number_2014 = question_y7_2014.Question_number_2014.str.replace('A','')


In [44]:
df2014_2 = pd.merge(df2014_1, question_y7_2014, left_on = 'Q', right_on = 'Question_number_2014',
        how = 'outer')

In [45]:
df2014_2.isnull().any()

Q                       False
Question_number_2014    False
Description_2014        False
Syllabus_2014           False
Verbs_2014              False
dtype: bool

In [46]:
# change the columns name
X_y7_2014.columns = df2014_2.Syllabus_2014

In [47]:
df2014_3 = X_y7_2014.T.reset_index().groupby(['Syllabus_2014']).apply(np.mean)

In [48]:
new_X_y7_2014 = df2014_3.T

In [49]:
new_X_y7_2014.shape

(194, 27)

In [50]:
new_X_y7_2014.head()

Syllabus_2014,MA2-11MG,MA2-13MG,MA2-14MG,MA2-15MG,MA2-17MG,MA2-19SP,MA2-4NA,MA2-6NA,MA2-8NA,MA3-11MG,...,MA4-10NA,MA4-12MG,MA4-13MG,MA4-15MG,MA4-17MG,MA4-19SP,MA4-4NA,MA4-5NA,MA4-7NA,MA4-9NA
SRN,,,,,,,,,,,,,,,,,,,,,
436556471,1.0,1.0,1.0,1.0,0.5,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.625,0.0,1.0,0.500000,1.00,1.0
434564891,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.875,0.0,0.5,0.500000,1.00,1.0
428415167,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.5,0.0,1.0,0.875,1.0,1.0,0.833333,1.00,1.0
432413110,1.0,1.0,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.375,0.0,0.0,0.166667,0.25,0.0
429541549,1.0,1.0,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.625,0.0,1.0,0.666667,0.50,1.0


# 2015


In [51]:
column_name = pd.Series(X_y7_2015.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2015_1 = pd.DataFrame(column_name,columns=['Q'])

In [52]:
question_y7_2015.Question_number_2015 = question_y7_2015.Question_number_2015.str.replace('-','')
question_y7_2015.Question_number_2015 = question_y7_2015.Question_number_2015.str.replace('A0','')
question_y7_2015.Question_number_2015 = question_y7_2015.Question_number_2015.str.replace('C0','C')
question_y7_2015.Question_number_2015 = question_y7_2015.Question_number_2015.str.replace('A','')


In [53]:
df2015_2 = pd.merge(df2015_1, question_y7_2015, left_on = 'Q', 
                    right_on = 'Question_number_2015',how = 'outer')

In [54]:
df2015_2.isnull().any()

Q                       False
Question_number_2015    False
Description_2015        False
Syllabus_2015           False
Verbs_2015              False
dtype: bool

In [55]:
# change the columns name
X_y7_2015.columns = df2015_2.Syllabus_2015

In [56]:
df2015_3 = X_y7_2015.T.reset_index().groupby(['Syllabus_2015']).apply(np.mean)

In [57]:
new_X_y7_2015 = df2015_3.T

In [58]:
new_X_y7_2015.head()

Syllabus_2015,MA1-11MG,MA1-8NA,MA2-11MG,MA2-13MG,MA2-15MG,MA2-16MG,MA2-17MG,MA2-6NA,MA2-7NA,MA3-10MG,...,MA3-8NA,MA3-9MG,MA4-10NA,MA4-12MG,MA4-17MG,MA4-21SP,MA4-4NA,MA4-5NA,MA4-7NA,MA4-8NA
SRN,,,,,,,,,,,,,,,,,,,,,
433786980,1.0,0.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,...,0.5,0.000000,0.5,1.0,1.0,1.0,1.0,1.00,0.0,1.000000
433787013,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.666667,1.0,1.0,0.5,1.0,0.5,0.75,1.0,0.333333
429541000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,...,0.5,0.333333,0.0,0.0,0.0,0.0,0.5,0.00,1.0,0.333333
432413128,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,0.0,1.0,0.0,0.0,0.5,0.25,0.0,0.000000
434665116,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.5,0.0,...,0.0,0.000000,0.5,0.0,0.5,1.0,0.5,0.00,0.0,0.000000


# 2016

In [59]:
X_y7_2016.dropna(inplace=True)

In [60]:
column_name = pd.Series(X_y7_2016.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2016_1 = pd.DataFrame(column_name,columns=['Q'])

In [61]:
question_y7_2016.Question_number_2016 = question_y7_2016.Question_number_2016.str.replace('-','')
question_y7_2016.Question_number_2016 = question_y7_2016.Question_number_2016.str.replace('A0','')
question_y7_2016.Question_number_2016 = question_y7_2016.Question_number_2016.str.replace('C0','C')
question_y7_2016.Question_number_2016 = question_y7_2016.Question_number_2016.str.replace('A','')


In [62]:
df2016_2 = pd.merge(df2016_1, question_y7_2016, left_on = 'Q', 
                    right_on = 'Question_number_2016',
        how = 'outer')

In [63]:
df2016_2.isnull().any()

Q                       False
Question_number_2016    False
Description_2016        False
Syllabus_2016           False
Verbs_2016              False
dtype: bool

In [64]:
X_y7_2016.columns = df2016_2.Syllabus_2016

In [65]:
df2016_3 = X_y7_2016.T.reset_index().groupby(['Syllabus_2016']).apply(np.mean)

In [66]:
new_X_y7_2016 = df2016_3.T
new_X_y7_2016.head()

Syllabus_2016,MA2-13MG,MA2-14MG,MA2-15MG,MA2-17MG,MA2-18SP,MA2-19SP,MA2-8NA,MA3-11MG,MA3-12MG,MA3-15MG,...,MA4-17MG,MA4-18MG,MA4-19SP,MA4-20SP,MA4-21SP,MA4-4NA,MA4-5NA,MA4-6NA,MA4-7NA,MA4-8NA
SRN,,,,,,,,,,,,,,,,,,,,,
435890202,0.0,0.0,1.0,1.0,1.0,1.0,0.5,0.666667,0.0,0.0,...,1.0,0.0,1.0,0.0,0.5,1.0,0.666667,1.0,0.333333,1.0
447613247,0.5,0.0,1.0,1.0,1.0,0.0,0.5,0.333333,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.333333,0.0,0.333333,0.5
435977111,1.0,0.0,1.0,1.0,1.0,1.0,0.5,0.666667,1.0,0.5,...,1.0,0.0,1.0,1.0,0.5,1.0,0.333333,0.0,0.333333,0.5
435890903,0.5,0.0,1.0,1.0,1.0,0.0,0.5,0.666667,1.0,0.5,...,0.0,0.0,1.0,0.0,1.0,0.0,0.666667,0.0,0.333333,0.5
435859500,1.0,0.0,1.0,1.0,1.0,1.0,0.5,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.666667,1.0,1.000000,1.0


# 2017

In [67]:
column_name = pd.Series(X_y7_2017.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2017_1 = pd.DataFrame(column_name,columns=['Q'])

In [68]:
question_y7_2017.Question_number_2017 = question_y7_2017.Question_number_2017.str.replace('-','')
question_y7_2017.Question_number_2017 = question_y7_2017.Question_number_2017.str.replace('A0','')
question_y7_2017.Question_number_2017 = question_y7_2017.Question_number_2017.str.replace('C0','C')
question_y7_2017.Question_number_2017 = question_y7_2017.Question_number_2017.str.replace('A','')


In [69]:
df2017_2 = pd.merge(df2017_1, question_y7_2017, left_on = 'Q', 
                    right_on = 'Question_number_2017',
        how = 'outer')

In [70]:
df2017_2.isnull().any()

Q                       False
Question_number_2017    False
Description_2017        False
Syllabus_2017           False
Verbs_2017              False
dtype: bool

In [71]:
X_y7_2017.columns = df2017_2.Syllabus_2017

In [72]:
df2017_3 = X_y7_2017.T.reset_index().groupby(['Syllabus_2017']).apply(np.mean)

In [73]:
new_X_y7_2017 = df2017_3.T
new_X_y7_2017.head()

Syllabus_2017,MA2-18SP,MA2-5NA,MA2-6NA,MA2-8NA,MA3-11MG,MA3-13MG,MA3-14MG,MA3-15MG,MA3-16MG,MA3-17MG,...,MA3-7NA,MA3-8NA,MA3-9MG,MA4-13MG,MA4-14MG,MA4-19SP,MA4-20SP,MA4-4NA,MA4-5NA,MA4-6NA
SRN,,,,,,,,,,,,,,,,,,,,,
441410735,1.0,0.0,1.0,1.000000,0.5,1.0,0.0,0.0,1.0,0.0,...,0.500000,1.000000,0.5,0.5,1.0,0.0,0.0,0.5,1.0,1.0
436808992,1.0,1.0,1.0,1.000000,0.5,0.5,0.0,0.0,1.0,1.0,...,0.833333,1.000000,1.0,0.5,1.0,0.0,0.0,1.0,1.0,1.0
436991703,1.0,0.0,1.0,1.000000,0.5,1.0,0.0,1.0,1.0,1.0,...,0.583333,0.666667,0.5,0.5,1.0,0.0,0.0,1.0,0.0,1.0
448607429,1.0,0.0,1.0,0.666667,0.5,0.5,0.0,0.0,0.0,0.0,...,0.583333,1.000000,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0
437076511,1.0,1.0,1.0,1.000000,0.5,0.5,1.0,1.0,1.0,1.0,...,1.000000,1.000000,0.5,0.5,1.0,1.0,0.0,1.0,1.0,1.0


In [76]:
#new_X_y7_2012.to_csv('X_y7_2012_2.csv', header=True, index = False)

In [77]:
#new_X_y7_2013.to_csv('X_y7_2013_2.csv', header=True, index = False)
#new_X_y7_2014.to_csv('X_y7_2014_2.csv', header=True, index = False)
#new_X_y7_2015.to_csv('X_y7_2015_2.csv', header=True, index = False)

In [27]:
#new_X_y7_2016.to_csv('X_y7_2016.csv', header=True, index = False)
#new_X_y7_2017.to_csv('X_y7_2017.csv', header=True, index = False)

# Other features: Verbs in the description of questions

Extract the verb from the each question

Group the same verbs and use the mean score for each verbs

## y7 2012 verbs

In [105]:
question_y7_2012.head()

,Question_number_2012,Description_2012,Syllabus_2012,Verbs_2012
0,NC1,3D: Determines the number of faces on an irreg...,MA2-14MG,Determines
1,C1,Frac and Dec: Solves a word problem involving ...,MA3-7NA,Solves
2,NC2,P and A: Calculates the value of a pronumeral ...,MA3-8NA,Calculates
3,C2,Integers: Calculates rise in temperature from ...,MA4-4NA,Calculates
4,NC3,Time: Calculates elapsed time in minutes,MA3-13MG,Calculates


In [106]:
X_y7_2012_v = y7_2012_y9_2014[['NM7(C1)', 'NM7(NC1)', 'NM7(C2)', 'NM7(NC2)', 'NM7(NC3)',
 'NM7(C3)', 'NM7(NC4)', 'NM7(C4)', 'NM7(NC5)', 'NM7(C5)', 'NM7(NC6)',
 'NM7(C6)', 'NM7(C7)', 'NM7(NC7)', 'NM7(NC8)', 'NM7(C8)', 'NM7(NC9)', 
 'NM7(C9)', 'NM7(C10)', 'NM7(NC10)', 'NM7(NC11)', 'NM7(C11)', 'NM7(C12)', 
 'NM7(NC12)', 'NM7(C13)', 'NM7(NC13)', 'NM7(NC14)', 'NM7(C14)', 'NM7(C15)',
 'NM7(NC15)', 'NM7(C16)', 'NM7(NC16)', 'NM7(C17)', 'NM7(NC17)', 'NM7(NC18)', 
 'NM7(C18)', 'NM7(NC19)', 'NM7(C19)', 'NM7(NC20)', 'NM7(C20)', 'NM7(NC21)', 
 'NM7(C21)', 'NM7(C22)', 'NM7(NC22)', 'NM7(NC23)', 'NM7(C23)', 'NM7(C24)', 
 'NM7(NC24)', 'NM7(NC25)', 'NM7(C25)', 'NM7(C26)', 'NM7(NC26)', 'NM7(C27)', 
 'NM7(NC27)', 'NM7(C28)', 'NM7(NC28)', 'NM7(NC29)', 'NM7(C29)', 'NM7(C30)', 
 'NM7(NC30)', 'NM7(C31)', 'NM7(NC31)', 'NM7(C32)', 'NM7(C32).1','SRN']].set_index('SRN')


In [107]:
question_y7_2012.Question_number_2012 = question_y7_2012.Question_number_2012.str.replace('-','')
question_y7_2012.Question_number_2012 = question_y7_2012.Question_number_2012.str.replace('A0','')
question_y7_2012.Question_number_2012 = question_y7_2012.Question_number_2012.str.replace('C0','C')
question_y7_2012.Question_number_2012 = question_y7_2012.Question_number_2012.str.replace('A','')

question_y7_2012.iloc[63].Question_number_2012 = 'C32.1'

column_name = pd.Series(X_y7_2012_v.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2012_1 = pd.DataFrame(column_name,columns=['Q'])
df2012_2 = pd.merge(df2012_1, question_y7_2012, left_on = 'Q', right_on = 'Question_number_2012',
        how = 'outer')

In [108]:
# rename the column names
X_y7_2012_v.columns = df2012_2.Verbs_2012
# group by for the same verbes
df2012_3 = X_y7_2012_v.T.reset_index().groupby(['Verbs_2012']).apply(np.mean).T

In [109]:
df2012_3.head()

Verbs_2012,Adds,Applies,Calculates,Compares,Continues,Converts,Describes,Determines,Estimates,Expresses,Finds,Generalises,Identifies,Interprets,Locates,Names,Performs,Rounds,Solves,Uses
SRN,,,,,,,,,,,,,,,,,,,,
436530790,1.0,0.666667,0.6250,0.0,0.0,0.0,1.0,1.000000,1.0,0.0,0.000000,0.0,0.615385,0.666667,0.0,1.0,1.0,1.0,0.500000,1.0
441546149,1.0,1.000000,0.8125,1.0,1.0,1.0,1.0,0.666667,1.0,1.0,0.500000,0.0,0.846154,1.000000,1.0,1.0,1.0,1.0,0.666667,1.0
432640256,1.0,0.000000,0.6875,0.0,0.0,0.0,1.0,1.000000,0.0,1.0,0.666667,0.0,0.769231,0.333333,0.0,1.0,0.0,0.0,0.666667,1.0
434220173,1.0,0.666667,0.5625,0.0,0.0,0.0,1.0,1.000000,0.0,0.0,0.333333,1.0,0.615385,0.333333,0.0,1.0,1.0,0.0,0.333333,0.5
431672294,1.0,1.000000,0.8125,0.0,1.0,1.0,1.0,1.000000,1.0,1.0,0.500000,0.0,0.923077,1.000000,0.0,1.0,1.0,1.0,0.500000,1.0


In [111]:
#df2012_3.to_csv('X_y7_2012_verbes_2.csv', header=True, index = False)

## y7 2013 verbes

In [112]:
X_y7_2013_v = y7_2013_y9_2015[['NM7(NC1)', 'NM7(C1)', 'NM7(C2)', 'NM7(NC2)', 'NM7(C3)', 'NM7(NC3)', 'NM7(NC4)', 'NM7(C4)', 
              'NM7(NC5)', 'NM7(C5)', 'NM7(NC6)', 'NM7(C6)', 'NM7(NC7)', 'NM7(C7)', 'NM7(C8)', 'NM7(NC8)', 
              'NM7(NC9)', 'NM7(C9)', 'NM7(NC10)', 'NM7(C10)', 'NM7(C11)', 'NM7(NC11)', 'NM7(C12)', 'NM7(NC12)', 
              'NM7(C13)', 'NM7(NC13)', 'NM7(NC14)', 'NM7(C14)', 'NM7(NC15)', 'NM7(C15)', 'NM7(C16)', 'NM7(NC16)', 
              'NM7(C17)', 'NM7(NC17)', 'NM7(NC18)', 'NM7(C18)', 'NM7(NC19)', 'NM7(C19)', 'NM7(NC20)', 'NM7(C20)', 
              'NM7(C21)', 'NM7(NC21)', 'NM7(NC22)', 'NM7(C22)', 'NM7(NC23)', 'NM7(C23)', 'NM7(C24)', 'NM7(NC24)', 
              'NM7(C25)', 'NM7(NC25)', 'NM7(C26)', 'NM7(NC26)', 'NM7(C27)', 'NM7(NC27)', 'NM7(C28)', 'NM7(NC28)', 
              'NM7(NC29)', 'NM7(C29)', 'NM7(NC30)', 'NM7(C30)', 'NM7(NC31)', 'NM7(C31)', 'NM7(NC32)', 'NM7(C32)','SRN']].set_index('SRN')


In [113]:
column_name = pd.Series(X_y7_2013_v.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2013_1 = pd.DataFrame(column_name,columns=['Q'])

question_y7_2013.Question_number_2013 = question_y7_2013.Question_number_2013.str.replace('-','')
question_y7_2013.Question_number_2013 = question_y7_2013.Question_number_2013.str.replace('A0','')
question_y7_2013.Question_number_2013 = question_y7_2013.Question_number_2013.str.replace('C0','C')
question_y7_2013.Question_number_2013 = question_y7_2013.Question_number_2013.str.replace('A','')


df2013_2 = pd.merge(df2013_1, question_y7_2013, left_on = 'Q', right_on = 'Question_number_2013',
        how = 'outer')

In [114]:
X_y7_2013_v.columns = df2013_2.Verbs_2013
df2013_3 = X_y7_2013_v.T.reset_index().groupby(['Verbs_2013']).apply(np.mean).T

In [115]:
df2013_3.head()

Verbs_2013,Adds,Applies,Calculates,Compares,Completes,Continues,Counts,Determines,Displays,Divides,...,Finds,Follows,Identifies,Interprets,Reads,Selects,Solves,Subtracts,Uses,Visualises
SRN,,,,,,,,,,,,,,,,,,,,,
436607369,1.0,0.0,0.6,0.0,0.50,0.0,0.0,0.625,1.0,0.5,...,0.666667,1.0,1.00,0.50,1.0,1.0,0.333333,1.0,1.0,1.0
429152038,1.0,1.0,0.8,1.0,1.00,1.0,1.0,0.750,1.0,1.0,...,0.888889,1.0,1.00,0.75,1.0,1.0,0.666667,1.0,1.0,1.0
434564981,1.0,1.0,0.8,1.0,1.00,1.0,1.0,0.875,1.0,1.0,...,0.777778,1.0,0.75,0.75,1.0,1.0,1.000000,1.0,1.0,1.0
428846257,1.0,0.0,0.7,1.0,1.00,1.0,0.0,0.875,1.0,0.5,...,0.555556,0.0,1.00,0.25,1.0,1.0,0.833333,1.0,1.0,1.0
429541042,0.0,1.0,0.7,1.0,0.75,1.0,0.0,0.750,1.0,1.0,...,0.777778,1.0,0.75,0.75,1.0,1.0,0.666667,1.0,1.0,1.0


In [116]:
#df2013_3.to_csv('X_y7_2013_verbes_2.csv', header=True, index = False)

## y7 2014 verbes

In [78]:
X_y7_2014_v = y7_2014_y9_2016[['NM7(C1)', 'NM7(NC1)', 'NM7(C2)', 'NM7(NC2)', 'NM7(C3)', 
                             'NM7(NC3)', 'NM7(NC4)', 'NM7(C4)', 'NM7(C5)', 'NM7(NC5)', 
                             'NM7(C6)', 'NM7(NC6)', 'NM7(C7)', 'NM7(NC7)', 'NM7(NC8)', 
                             'NM7(C8)', 'NM7(C9)', 'NM7(NC9)', 'NM7(C10)', 'NM7(NC10)', 
                             'NM7(NC11)', 'NM7(C11)', 'NM7(NC12)', 'NM7(C12)', 'NM7(NC13)', 'NM7(C13)', 'NM7(NC14)', 'NM7(C14)', 'NM7(NC15)', 'NM7(C15)', 'NM7(C16)', 'NM7(NC16)', 'NM7(NC17)', 'NM7(C17)', 'NM7(NC18)', 'NM7(C18)', 'NM7(C19)', 'NM7(NC19)', 'NM7(NC20)', 'NM7(C20)', 'NM7(NC21)', 'NM7(C21)', 'NM7(NC22)', 'NM7(C22)', 'NM7(NC23)', 'NM7(C23)', 'NM7(C24)', 'NM7(NC24)', 'NM7(NC25)', 'NM7(C25)', 'NM7(NC26)', 'NM7(C26)', 'NM7(NC27)', 'NM7(C27)', 'NM7(C28)', 'NM7(NC28)', 'NM7(C29)', 'NM7(NC29)', 'NM7(C30)', 
                             'NM7(NC30)', 'NM7(C31)', 'NM7(NC31)', 'NM7(NC32)', 'NM7(C32)','SRN']].set_index('SRN')


In [79]:
column_name = pd.Series(X_y7_2014_v.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2014_1 = pd.DataFrame(column_name,columns=['Q'])
question_y7_2014.Question_number_2014 = question_y7_2014.Question_number_2014.str.replace('-','')
question_y7_2014.Question_number_2014 = question_y7_2014.Question_number_2014.str.replace('A0','')
question_y7_2014.Question_number_2014 = question_y7_2014.Question_number_2014.str.replace('C0','C')
question_y7_2014.Question_number_2014 = question_y7_2014.Question_number_2014.str.replace('A','')
df2014_2 = pd.merge(df2014_1, question_y7_2014, left_on = 'Q', right_on = 'Question_number_2014',
        how = 'outer')

In [80]:
# change the columns name
X_y7_2014_v.columns = df2014_2.Verbs_2014
df2014_3 = X_y7_2014_v.T.reset_index().groupby(['Verbs_2014']).apply(np.mean).T

In [81]:
df2014_3.head()

Verbs_2014,Calculates,Determines,Extends,Finds,Follows,Identifies,Interprets,Locates,Matches,Reads,Recognises,Solves,Uses,Writes
SRN,,,,,,,,,,,,,,
436556471,0.500000,1.0,1.0,0.333333,0.0,0.81250,0.50,1.0,0.0,0.5,0.666667,0.571429,1.0,0.0
434564891,0.833333,1.0,1.0,0.666667,1.0,0.84375,0.50,1.0,1.0,1.0,0.666667,0.714286,1.0,1.0
428415167,0.833333,1.0,1.0,0.666667,0.0,0.87500,0.75,1.0,1.0,1.0,1.000000,0.714286,1.0,0.0
432413110,0.166667,0.0,0.0,0.000000,0.0,0.25000,0.00,0.0,0.0,0.5,0.333333,0.142857,1.0,0.0
429541549,0.500000,1.0,1.0,0.333333,1.0,0.81250,0.50,1.0,1.0,1.0,0.333333,0.571429,0.0,0.0


In [83]:
#df2014_3.to_csv('X_y7_2014_verbes_2.csv', header=True, index = False)

## y7 2015 verbes

In [84]:
X_y7_2015_v = y7_2015_y9_2017[['NM7(C1)', 'NM7(NC1)', 'NM7(NC2)', 'NM7(C2)', 'NM7(NC3)', 
                             'NM7(C3)', 'NM7(C4)', 'NM7(NC4)', 'NM7(NC5)', 'NM7(C5)', 
                             'NM7(NC6)', 'NM7(C6)', 'NM7(C7)', 'NM7(NC7)', 'NM7(C8)', 
                             'NM7(NC8)', 'NM7(NC9)', 'NM7(C9)', 'NM7(NC10)', 'NM7(C10)',
                             'NM7(C11)', 'NM7(NC11)', 'NM7(NC12)', 'NM7(C12)', 'NM7(C13)', 
                             'NM7(NC13)', 'NM7(C14)', 'NM7(NC14)', 'NM7(NC15)', 'NM7(C15)',
                             'NM7(C16)', 'NM7(NC16)', 'NM7(C17)', 'NM7(NC17)', 'NM7(NC18)',
                             'NM7(C18)', 'NM7(NC19)', 'NM7(C19)', 'NM7(NC20)', 'NM7(C20)', 
                             'NM7(NC21)', 'NM7(C21)', 'NM7(NC22)', 'NM7(C22)', 'NM7(NC23)', 
                             'NM7(C23)', 'NM7(C24)', 'NM7(NC24)', 'NM7(NC25)', 'NM7(C25)', 
                             'NM7(NC26)', 'NM7(C26)', 'NM7(NC27)', 'NM7(C27)', 'NM7(NC28)',
                             'NM7(C28)', 'NM7(NC29)', 'NM7(C29)', 'NM7(C30)', 
                             'NM7(NC30)', 'NM7(NC31)', 'NM7(C31)', 'NM7(NC32)', 'NM7(C32)','SRN']].set_index('SRN')


In [85]:
column_name = pd.Series(X_y7_2015_v.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2015_1 = pd.DataFrame(column_name,columns=['Q'])
question_y7_2015.Question_number_2015 = question_y7_2015.Question_number_2015.str.replace('-','')
question_y7_2015.Question_number_2015 = question_y7_2015.Question_number_2015.str.replace('A0','')
question_y7_2015.Question_number_2015 = question_y7_2015.Question_number_2015.str.replace('C0','C')
question_y7_2015.Question_number_2015 = question_y7_2015.Question_number_2015.str.replace('A','')
df2015_2 = pd.merge(df2015_1, question_y7_2015, left_on = 'Q', 
                    right_on = 'Question_number_2015',how = 'outer')

In [86]:
# change the columns name
X_y7_2015_v.columns = df2015_2.Verbs_2015
df2015_3 = X_y7_2015_v.T.reset_index().groupby(['Verbs_2015']).apply(np.mean).T

In [87]:
df2015_3.head()

Verbs_2015,Applies,Calculates,Chooses,Describes,Determines,Estimates,Expresses,Finds,Formulates,Identifies,Interprets,Orders,Selects,Solves,Uses
SRN,,,,,,,,,,,,,,,
433786980,1.0,0.785714,1.0,1.0,0.714286,1.000000,1.0,1.0,0.666667,0.727273,0.8,1.0,0.666667,0.25,0.714286
433787013,0.0,0.857143,1.0,1.0,0.714286,0.666667,1.0,1.0,0.666667,0.727273,0.4,1.0,1.000000,0.75,0.714286
429541000,0.0,0.142857,0.0,1.0,0.285714,0.666667,0.0,0.5,0.333333,0.090909,0.0,0.0,0.333333,0.50,0.142857
432413128,0.0,0.214286,0.0,1.0,0.142857,0.333333,1.0,0.5,0.000000,0.272727,0.2,0.0,0.333333,0.00,0.142857
434665116,0.0,0.285714,0.0,1.0,0.000000,0.666667,0.0,0.0,0.000000,0.636364,0.2,0.0,0.666667,0.00,0.142857


In [89]:
#df2015_3.to_csv('X_y7_2015_verbes_2.csv', header=True, index = False)

## y7 2016 verbes

In [118]:
X_y7_2016_v = y7_2016[['NM7(C1)', 'NM7(NC1)', 'NM7(NC2)', 'NM7(C2)', 'NM7(NC3)', 
                     'NM7(C3)', 'NM7(NC4)', 'NM7(C4)', 'NM7(C5)', 'NM7(NC5)', 
                     'NM7(NC6)', 'NM7(C6)', 'NM7(C7)', 'NM7(NC7)', 'NM7(NC8)', 
                     'NM7(C8)', 'NM7(NC9)', 'NM7(C9)', 'NM7(NC10)', 'NM7(C10)', 
                     'NM7(C11)', 'NM7(NC11)', 'NM7(NC12)', 'NM7(C12)', 'NM7(NC13)',
                     'NM7(C13)', 'NM7(C14)', 'NM7(NC14)', 'NM7(NC15)', 'NM7(C15)',
                     'NM7(C16)', 'NM7(NC16)', 'NM7(NC17)', 'NM7(C17)', 
                     'NM7(NC18)', 'NM7(C18)', 'NM7(NC19)', 'NM7(C19)', 'NM7(NC20)',
                     'NM7(C20)', 'NM7(NC21)', 'NM7(C21)', 'NM7(NC22)', 'NM7(C22)',
                     'NM7(C23)', 'NM7(NC23)', 'NM7(NC24)', 'NM7(C24)', 'NM7(C25)', 
                     'NM7(NC25)', 'NM7(C26)', 'NM7(NC26)', 'NM7(C27)', 'NM7(NC27)', 
                     'NM7(NC28)', 'NM7(C28)', 'NM7(NC29)', 'NM7(C29)', 'NM7(NC30)',
        
                     'NM7(C30)', 'NM7(NC31)', 'NM7(C31)', 'NM7(C32)', 'NM7(NC32)','SRN']].set_index('SRN')
X_y7_2016_v.dropna(inplace=True)

In [119]:
column_name = pd.Series(X_y7_2016_v.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2016_1 = pd.DataFrame(column_name,columns=['Q'])
question_y7_2016.Question_number_2016 = question_y7_2016.Question_number_2016.str.replace('-','')
question_y7_2016.Question_number_2016 = question_y7_2016.Question_number_2016.str.replace('A0','')
question_y7_2016.Question_number_2016 = question_y7_2016.Question_number_2016.str.replace('C0','C')
question_y7_2016.Question_number_2016 = question_y7_2016.Question_number_2016.str.replace('A','')
df2016_2 = pd.merge(df2016_1, question_y7_2016, left_on = 'Q', 
                    right_on = 'Question_number_2016',
        how = 'outer')

In [120]:
df2016_2.head()

,Q,Question_number_2016,Description_2016,Syllabus_2016,Verbs_2016
0,C1,C1,Multiplication and Division 2: Solves a multi-...,MA3-6NA,Solves
1,NC1,NC1,Position: Uses compass directions to identify ...,MA2-17MG,Uses
2,NC2,NC2,Patterns and Algebra 2: Uses properties of eve...,MA2-8NA,Uses
3,C2,C2,Ratios and Rates: Uses a ratio to solve a prob...,MA4-7NA,Uses
4,NC3,NC3,Fractions and Decimals 2: Calculates a fractio...,MA3-7NA,Calculates


In [121]:
X_y7_2016_v.columns = df2016_2.Verbs_2016
df2016_3 = X_y7_2016_v.T.reset_index().groupby(['Verbs_2016']).apply(np.mean).T

In [122]:
df2016_3.head()

Verbs_2016,Adds,Calculates,Classifies,Compares,Continues,Converts,Describes,Determines,Divides,Estimates,Extends,Identifies,Interprets,Locates,Multiplies,Reasons,Recognises,Selects,Solves,Uses
SRN,,,,,,,,,,,,,,,,,,,,
435890202,1.0,0.4,1.0,0.0,0.0,1.000000,0.0,0.333333,0.5,1.0,0.0,0.714286,0.0,1.0,0.333333,0.0,0.0,0.75,0.333333,0.8
447613247,0.5,0.2,0.0,0.5,1.0,0.333333,0.5,0.444444,0.5,0.0,0.0,0.285714,0.5,0.0,0.000000,0.0,0.0,0.25,0.333333,0.8
435977111,1.0,0.6,1.0,0.0,1.0,0.333333,1.0,0.444444,0.5,0.0,0.0,0.714286,1.0,0.0,0.666667,0.0,0.0,0.75,0.666667,0.8
435890903,0.0,0.5,0.0,0.5,0.0,0.666667,0.0,0.333333,0.0,1.0,0.0,0.428571,0.5,0.0,0.333333,1.0,0.0,0.50,0.500000,0.8
435859500,1.0,0.7,1.0,1.0,1.0,1.000000,0.0,0.666667,0.5,1.0,0.0,0.857143,0.5,1.0,0.000000,1.0,1.0,0.75,0.833333,1.0


In [123]:
df2016_3.shape

(206, 20)

In [124]:
#df2016_3.to_csv('X_y7_2016_verbes_2.csv', header=True, index = False)

## y7 2017 verbes

In [125]:
X_y7_2017_v = y7_2017[['NM7(C1)', 'NM7(NC1)', 'NM7(C2)', 
                     'NM7(NC2)', 'NM7(C3)', 'NM7(NC3)', 
                     'NM7(NC4)', 'NM7(C4)', 'NM7(C5)', 
                     'NM7(NC5)', 'NM7(NC6)', 'NM7(C6)', 'NM7(NC7)', 'NM7(C7)', 'NM7(NC8)', 
                     'NM7(C8)', 'NM7(C9)', 'NM7(C10)', 'NM7(C11)', 'NM7(C12)', 'NM7(C13)', 
                     'NM7(C14)', 'NM7(C15)', 'NM7(C16)', 'NM7(C17)', 'NM7(C18)',
                     'NM7(C19)', 'NM7(C20)', 'NM7(C21)', 'NM7(C22)', 'NM7(C23)',
                     'NM7(C24)', 'NM7(C25)', 'NM7(C26)', 'NM7(C27)', 'NM7(C28)', 
                     
                     'NM7(C29)', 'NM7(C30)', 'NM7(C31)', 'NM7(C32)', 'NM7(C33)', 
                     'NM7(C34)', 'NM7(C35)', 'NM7(C36)', 'NM7(C37)', 'NM7(C38)', 
                     'NM7(C39)', 'NM7(C40)','SRN']].set_index('SRN')

In [126]:
column_name = pd.Series(X_y7_2017_v.columns)
column_name = column_name.str.replace('NM7\(','')
column_name = column_name.str.replace('\)','')
df2017_1 = pd.DataFrame(column_name,columns=['Q'])
question_y7_2017.Question_number_2017 = question_y7_2017.Question_number_2017.str.replace('-','')
question_y7_2017.Question_number_2017 = question_y7_2017.Question_number_2017.str.replace('A0','')
question_y7_2017.Question_number_2017 = question_y7_2017.Question_number_2017.str.replace('C0','C')
question_y7_2017.Question_number_2017 = question_y7_2017.Question_number_2017.str.replace('A','')
df2017_2 = pd.merge(df2017_1, question_y7_2017, left_on = 'Q', 
                    right_on = 'Question_number_2017',
        how = 'outer')

In [127]:
df2017_2.head()

,Q,Question_number_2017,Description_2017,Syllabus_2017,Verbs_2017
0,C1,C1,Identifies the category with the greatest tota...,MA2-18SP,Identifies
1,NC1,NC1,Solves a problem using the division facts of five,MA2-6NA,Solves
2,C2,C2,Converts a time from the 24-hour system to the...,MA3-13MG,Converts
3,NC2,NC2,Adds decimals in context,MA3-7NA,Adds
4,C3,C3,Identifies the smallest decimal from a list,MA3-7NA,Identifies


In [128]:
X_y7_2017_v.columns = df2017_2.Verbs_2017
df2017_3 = X_y7_2017_v.T.reset_index().groupby(['Verbs_2017']).apply(np.mean).T

In [129]:
df2017_3.head()

Verbs_2017,Adds,Calculates,Chooses,Compares,Converts,Describes,Determines,Divides,Estimates,Evaluates,...,Interprets,Makes,Multiplies,Recognises,Represents,Selects,Solves,Substitutes,Subtracts,Uses
SRN,,,,,,,,,,,,,,,,,,,,,
441410735,1.0,0.333333,1.0,0.0,1.000000,1.0,0.75,0.0,1.0,1.0,...,0.5,0.0,1.0,1.0,0.0,1.0,0.50,1.0,0.000000,0.4
436808992,1.0,0.666667,1.0,1.0,0.333333,1.0,1.00,0.0,1.0,0.5,...,0.0,0.0,1.0,1.0,0.0,1.0,0.50,1.0,1.000000,0.8
436991703,1.0,0.333333,1.0,0.0,0.666667,1.0,1.00,0.0,1.0,0.5,...,0.5,0.0,1.0,1.0,1.0,1.0,0.75,0.0,0.666667,0.8
448607429,1.0,0.000000,1.0,1.0,0.333333,1.0,0.75,0.0,0.0,0.0,...,0.0,0.0,0.5,1.0,1.0,0.0,0.50,1.0,0.666667,0.4
437076511,1.0,0.666667,1.0,0.0,0.333333,1.0,1.00,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.00,1.0,1.000000,1.0


In [130]:
#df2017_3.to_csv('X_y7_2017_verbes_2.csv', header=True, index = False)